In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

# Preprocessing:

In [2]:
from datasets import load_dataset

# Load the SQUAD dataset
raw_datasets = load_dataset("squad")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
raw_datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [4]:
from transformers import AutoTokenizer, TFDistilBertForQuestionAnswering

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True) # Setting use_fast=True


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
def tokenize_function(example):
    tokenized = tokenizer(example["question"], example["context"])

    # Tokenize context separately to find answer positions
    context_tokenized = tokenizer(example["context"], add_special_tokens=False)

    filtered_data = {key: [] for key in tokenized.keys()}  # Store valid examples
    filtered_data["start_positions"] = []
    filtered_data["end_positions"] = []

    for i in range(len(example["question"])):  # Loop over batch
        if len(tokenized["input_ids"][i]) > tokenizer.model_max_length:
            continue  # Skip this example properly

        start_token_pos = context_tokenized.char_to_token(i, example["answers"][i]["answer_start"][0])
        end_token_pos = context_tokenized.char_to_token(i, example["answers"][i]["answer_start"][0] + len(example["answers"][i]["text"][0]) - 1)

        if start_token_pos is None or end_token_pos is None:
            continue  # Skip invalid examples properly

        # Find first [SEP] token index
        sep_token_pos = tokenized["input_ids"][i].index(tokenizer.sep_token_id)

        # Adjust answer positions
        start_token_pos += sep_token_pos + 1
        end_token_pos += sep_token_pos + 1

        # Append only valid examples
        for key in tokenized.keys():
            filtered_data[key].append(tokenized[key][i])

        filtered_data["start_positions"].append(start_token_pos)
        filtered_data["end_positions"].append(end_token_pos)

    return filtered_data  # Only return correctly filtered data

# Apply mapping
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets['train'].column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 87464
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10511
    })
})

In [7]:
from tqdm import tqdm

Checking all labels are of int:

In [8]:
for i in tqdm(range(len(tokenized_datasets['train']))):
  if isinstance(tokenized_datasets['train'][i]['start_positions'], int) and isinstance(tokenized_datasets['train'][i]['end_positions'], int):
    continue
  else:
    raise ValueError(tokenized_datasets['train'][i]['start_positions'], tokenized_datasets['train'][i]['end_positions'])


100%|██████████| 87464/87464 [00:42<00:00, 2060.12it/s]


In [9]:
from transformers import DataCollatorWithPadding

#for variable length padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [10]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["start_positions", "end_positions"],  # Include both labels
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)

In [11]:
tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["start_positions", "end_positions"],  # Include both labels
    shuffle=False,
    collate_fn=data_collator,
    batch_size=16,
)

# Training:

In [12]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [13]:
model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  65190912  
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 65192450 (248.69 MB)
Trainable params: 65192450 (248.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
#colab work better with tf_keras
import tf_keras

In [15]:
from tf_keras.optimizers.schedules import PolynomialDecay
from tf_keras.optimizers import Adam

batch_size = 16
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

In [16]:
# Compile the model
model.compile(
    optimizer=opt,
    loss=tf_keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3
)

In [ ]:
#saving trained model and tokenizer
model.save_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")

('/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model/tokenizer_config.json',
 '/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model/special_tokens_map.json',
 '/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model/vocab.txt',
 '/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model/added_tokens.json',
 '/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model/tokenizer.json')

# Testing the model and their performance

In [5]:
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer

#loading trained model
model = TFAutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at /content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [25]:
import tensorflow as tf

In [15]:
import numpy as np

In [9]:
predictions = model.predict(tf_validation_dataset)

657/657 [==============================] - 96s 136ms/step


In [ ]:
# Extract start and end logits from predictions
start_logits = predictions.start_logits
end_logits = predictions.end_logits

# Save as NumPy arrays for saving computation during testing
np.save("/content/drive/MyDrive/Q_A_chatbot/start_logits_validation_hf_bert.npy", start_logits)
np.save("/content/drive/MyDrive/Q_A_chatbot/end_logits_validation_hf_bert.npy", end_logits)

In [16]:
# Load from NumPy
start_logits_data = np.load("/content/drive/MyDrive/Q_A_chatbot/start_logits_validation_hf_bert.npy", allow_pickle=True)
end_logits_data = np.load("/content/drive/MyDrive/Q_A_chatbot/end_logits_validation_hf_bert.npy", allow_pickle=True)

In [23]:
start_logits = start_logits_data.item()
end_logits = end_logits_data.item()

In [21]:
start_logits = start_logits.to_tensor()
end_logits = end_logits.to_tensor()

In [46]:
from sklearn.metrics import f1_score

start_preds = tf.argmax(start_logits, axis=-1)
end_preds = tf.argmax(end_logits, axis=-1)

true_start_positions = []
true_end_positions = []

for batch in tqdm(tf_validation_dataset):
  true_start_positions.extend(batch[1]["start_positions"].numpy())
  true_end_positions.extend(batch[1]["end_positions"].numpy())

accuracy = tf.reduce_mean(tf.cast(tf.equal(start_preds, true_start_positions), dtype=tf.float32))
print(f"Accuracy: {accuracy.numpy()}")

f1_start = f1_score(true_start_positions, start_preds.numpy(), average='weighted')
f1_end = f1_score(true_end_positions, end_preds.numpy(), average='weighted')
print(f"F1 Score (Start): {f1_start}")
print(f"F1 Score (End): {f1_end}")

# Calculate an overall F1 score
f1_overall = (f1_start + f1_end) / 2
print(f"Overall F1 Score: {f1_overall}")


Accuracy: 0.6522690653800964
F1 Score (Start): 0.6574699156496815
F1 Score (End): 0.6930416691787269
Overall F1 Score: 0.6752557924142042


**Observation:** Getting the Overall F1 Score of 0.6752557924142042 which is good starting point but it still need further improvements.

## Now finding the right threshold value which give us better accuracy:

In [22]:
start_preds = tf.argmax(start_logits, axis=-1)
end_preds = tf.argmax(end_logits, axis=-1)

In [32]:
probabilities = []
probabilities.extend(tf.nn.softmax(start_logits, axis=-1))

In [37]:
from tqdm import tqdm

In [38]:
true_start_positions = []
true_end_positions = []

# Extract true positions and probabilities
for batch in tqdm(tf_validation_dataset):
    true_start_positions.extend(batch[1]["start_positions"])
    true_end_positions.extend(batch[1]["end_positions"])

100%|██████████| 657/657 [00:16<00:00, 39.23it/s]


In [40]:
(true_start_positions)[:3]

[<tf.Tensor: shape=(), dtype=int64, numpy=46>,
 <tf.Tensor: shape=(), dtype=int64, numpy=57>,
 <tf.Tensor: shape=(), dtype=int64, numpy=89>]

In [65]:
threshold = 0.9
filtered_indices = [i for i, prob in tqdm(enumerate(probabilities)) if tf.reduce_max(prob) > threshold]

10511it [00:04, 2254.59it/s]


In [66]:
len(filtered_indices)

4824

In [67]:
filtered_true_start_positions = [true_start_positions[i] for i in filtered_indices]

In [68]:
filtered_start_preds = [start_preds[i] for i in filtered_indices]

In [69]:
filtered_start_preds[:4]

[<tf.Tensor: shape=(), dtype=int64, numpy=57>,
 <tf.Tensor: shape=(), dtype=int64, numpy=72>,
 <tf.Tensor: shape=(), dtype=int64, numpy=73>,
 <tf.Tensor: shape=(), dtype=int64, numpy=157>]

In [70]:
filtered_accuracy = tf.reduce_mean(tf.cast(tf.equal(filtered_start_preds, filtered_true_start_positions), dtype=tf.float32))
print(f"Accuracy (prob > {threshold}): {filtered_accuracy.numpy()}")


Accuracy (prob > 0.9): 0.8884742856025696


**Observation:** With threshold 0.9 accuracy is 0.8884742856025696 but more than 50% of data get filtered so the best threshold is 0.6 with accuracy 0.710823...% .

# Inference:

In [60]:
question = "who was the incharge of The Apollo 11 mission"
text = """The Apollo 11 mission, launched on July 16, 1969, was the first human-crewed mission to land on the Moon.
Commanded by Neil Armstrong, with Buzz Aldrin and Michael Collins, the mission was historic but not without its challenges.
During the journey, the spacecraft encountered minor navigation errors that required mid-course corrections.
While approaching the lunar surface, the Lunar Module 'Eagle' experienced an overloaded computer due to multiple radar signals.
This issue nearly forced mission control to abort the landing. However, NASA engineers and the crew quickly analyzed the problem and proceeded cautiously.
As Armstrong piloted the lander manually, he realized that the pre-selected landing site was filled with large boulders and craters, making it unsafe.
With fuel running low, he had to find an alternative landing spot within seconds. He skillfully guided the spacecraft to a safer area and touched down with only about 20 seconds of fuel remaining.
Further challenges arose with the astronaut's movement on the lunar surface. The Moon's lower gravity made it difficult to walk naturally, requiring careful adaptation.
Additionally, the fine lunar dust clung to their suits, making visibility and equipment maintenance harder.
After spending over 21 hours on the Moon, Armstrong and Aldrin re-entered the Lunar Module and prepared for ascent.
Another major challenge emerged when they discovered that a switch for the ascent engine had broken off.
Without this switch, the engine would not ignite. In a critical moment, Aldrin used a simple felt-tip pen to activate the switch, allowing the module to lift off safely.
Finally, re-entry into Earth’s atmosphere required precise calculations to ensure the spacecraft would not bounce off or burn up due to excessive heat.
The crew successfully splashed down in the Pacific Ocean on July 24, where they were recovered by the USS Hornet and subsequently placed in quarantine to prevent potential contamination from lunar material.
Despite these challenges, Apollo 11 completed its mission, marking one of the greatest achievements in human history and paving the way for future lunar explorations."""


# Tokenize without 'token_type_ids'
inputs = tokenizer(question, text, return_tensors="tf")
inputs.pop("token_type_ids", None)  # Remove token_type_ids if present

outputs = model(**inputs)

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'Neil Armstrong'

# Gradio Deploy

In [18]:
!pip install huggingface_hub

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login

In [22]:
import os

In [23]:
from huggingface_hub import HfApi

# uploading the finetuned model to huggingFace space
api = HfApi(token=os.getenv("tokens here"))
api.upload_folder(
    folder_path="/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model",
    repo_id="iamgulshan/bert-qa-squad",
    repo_type="model",
)


tf_model.h5:   0%|          | 0.00/261M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iamgulshan/bert-qa-squad/commit/336cd966dfd825abfb9132cfe487cf353eba8e14', commit_message='Upload folder using huggingface_hub', commit_description='', oid='336cd966dfd825abfb9132cfe487cf353eba8e14', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iamgulshan/bert-qa-squad', endpoint='https://huggingface.co', repo_type='model', repo_id='iamgulshan/bert-qa-squad'), pr_revision=None, pr_num=None)

In [71]:
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer

model = TFAutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model")

Some layers from the model checkpoint at /content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at /content/drive/MyDrive/Q_A_chatbot/my_hf_bert_qa_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import numpy as np

In [79]:

import gradio as gr

def qa_model(question, context):
    inputs = tokenizer(question, context, return_tensors="tf")
    inputs.pop("token_type_ids", None)  # Remove token_type_ids if present
    outputs = model(**inputs)

    answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])


    answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
    start_prob = tf.nn.softmax(outputs.start_logits, axis=-1)[0][answer_start_index]
    end_prob = tf.nn.softmax(outputs.end_logits, axis=-1)[0][answer_end_index]

    avg_prob = (start_prob + end_prob) / 2

    if avg_prob >= 0.5:
      predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
      return tokenizer.decode(predict_answer_tokens)
    else:
      return "Sorry! I am not sure about the answer :)"

In [77]:
question = "what is your name?"
context = "My name is gulshan and i am from patna."

In [80]:
qa_model(question, context)

'gulshan'

In [ ]:

iface = gr.Interface(
    fn=qa_model,
    inputs=["text", "text"],
    outputs="text",
    title="Question Answering Model",
    description="Ask a question and provide a context to get an answer.",
)

iface.launch()


**The end**